In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from statistics import mean
from scipy import stats

In [ ]:
results_folder = os.getcwd()
csv_details = "EXPERIMENTS.csv"
csv_explicit = "explicit.csv"
csv_implicit = "implicit.csv"
csv_init = "init.csv"

folder_plots = "plots/godspeed/"

In [ ]:
exp_details = pd.read_csv(results_folder+"/"+csv_details)
survey_explicit = pd.read_csv(results_folder+"/"+csv_explicit)
survey_implicit = pd.read_csv(results_folder+"/"+csv_implicit)

# Isolate columns that we want
# In surveys, Godspeed is from column 27 (starting from 0).
for i in range(27):
    survey_implicit.drop(survey_implicit.columns[[0]], axis=1, inplace=True)
    survey_explicit.drop(survey_explicit.columns[[0]], axis=1, inplace=True)

In [ ]:
sections = ["anthropomorphism", "animacy", "likeability", "intelligence", "safety",
           "overall", "sysuse", "infoqual", "interqual"]

# CREATE GODSPEED CSV

In [ ]:
def get_mean_godspeed(df_survey, from_col, to_col):
    mean_godspeed = []
    for index in range(len(df_survey)):
        all_values = df_survey.iloc[index].tolist()
        godspeed_index = all_values[from_col:to_col] # 5, 6 or 3 items depending on section
        mean_godspeed.append(np.round(mean(godspeed_index), 2))
    return mean_godspeed

OTHER METRICS:

OVERALL: Average responses to 1 - 19

System Usefulness (SYSUSE): average responses to 1 - 8

Information Quality (INFOQUAL): average responses to 9 - 15

Interface Quality (INTERQUAL): average responses to 16 - 18

In [ ]:
godspeed_implicit = {}
godspeed_explicit = {}
# Anthropomorphism
godspeed_implicit["anthropomorphism"] = get_mean_godspeed(survey_implicit, 0, 5)
godspeed_explicit["anthropomorphism"] = get_mean_godspeed(survey_explicit, 0, 5)

#Animacy
godspeed_implicit["animacy"] = get_mean_godspeed(survey_implicit, 5, 11)
godspeed_explicit["animacy"] = get_mean_godspeed(survey_explicit, 5, 11)

#Likeability
godspeed_implicit["likeability"] = get_mean_godspeed(survey_implicit, 11, 16)
godspeed_explicit["likeability"] = get_mean_godspeed(survey_explicit, 11, 16)

#Perceived intelligence
godspeed_implicit["intelligence"] = get_mean_godspeed(survey_implicit, 16, 21)
godspeed_explicit["intelligence"] = get_mean_godspeed(survey_explicit, 16, 21)

#Perceived safety
godspeed_implicit["safety"] = get_mean_godspeed(survey_implicit, 21, 24)
godspeed_explicit["safety"] = get_mean_godspeed(survey_explicit, 21, 24)

# OVERALL: Average responses to 1 - 19 [0-18]
godspeed_implicit["overall"] = get_mean_godspeed(survey_implicit, 0, 19)
godspeed_explicit["overall"] = get_mean_godspeed(survey_explicit, 0, 19)

# System Usefulness (SYSUSE): average responses to 1 - 8 [0-7]
godspeed_implicit["sysuse"] = get_mean_godspeed(survey_implicit, 0, 8)
godspeed_explicit["sysuse"] = get_mean_godspeed(survey_explicit, 0, 8)

# Information Quality (INFOQUAL): average responses to 9 - 15 [8-14]
godspeed_implicit["infoqual"] = get_mean_godspeed(survey_implicit, 8, 14)
godspeed_explicit["infoqual"] = get_mean_godspeed(survey_explicit, 8, 14)

# Interface Quality (INTERQUAL): average responses to 16 - 18 [15-17]
godspeed_implicit["interqual"] = get_mean_godspeed(survey_implicit, 15, 18)
godspeed_explicit["interqual"] = get_mean_godspeed(survey_explicit, 15, 18)


### If we want to consider the order of questionnaires submission

In [ ]:
# Create list of first and second surveys (explicit, implicit)
first_surveys = []
second_surveys = []
for index in np.unique(exp_details['ID']):
    order_behaviors = exp_details[exp_details['ID'] == index]['BEHAVIOR'].tolist()
    first_surveys.append(order_behaviors[0])
    second_surveys.append(order_behaviors[1])

# Create dataframe
df = pd.DataFrame(np.unique(exp_details['ID']), columns = ['ID'] )
df['FIRST_SURVEY'] = first_surveys
df['SECOND_SURVEY'] = second_surveys

# ADD sections
for section in sections:
    first_values = []
    second_values = []
    for i in range(len(first_surveys)):
        if first_surveys[i] == 'explicit' and second_surveys[i] == 'implicit':
            first_values.append(godspeed_explicit[section][i])
            second_values.append(godspeed_implicit[section][i])
        elif first_surveys[i] == 'implicit' and second_surveys[i] == 'explicit':
            first_values.append(godspeed_implicit[section][i])
            second_values.append(godspeed_explicit[section][i])
        else:
            print("ERROR: in explicit or implicit trust sequence.")
    df['FIRST_'+section.upper()] = first_values
    df['SECOND_'+section.upper()] = second_values
    
# Save csv
#df.to_csv("results_godspeed.csv", index=False)


### If we want to consider only type of questionnaire referring to the behavior (explicit, implicit)

In [ ]:
# Create dataframe
df = pd.DataFrame(np.unique(exp_details['ID']), columns = ['ID'] )

# ADD sections
for section in sections:
    df['E_'+section.upper()] = godspeed_explicit[section]
    df['I_'+section.upper()] = godspeed_implicit[section]

# Save csv
df.to_csv("results_godspeed.csv", index=False)

# ANALYSIS

In [ ]:
df = pd.read_csv("results_godspeed.csv")
#df.drop(df[df['ID'] == 59].index, inplace=True)
df

In [ ]:
statistics = pd.DataFrame(sections, columns = ['SECTION'] )

In [ ]:
def update_statistics(df, indexes, behavior, title):
    for section in sections:
        if behavior == "explicit":
            col = df['E_'+section.upper()][indexes].tolist()
        else:
            col = df['I_'+section.upper()][indexes].tolist()
            
        row = statistics[statistics['SECTION'] == section].index
        statistics.loc[row, 'MEAN_'+title] = np.round(mean(col), 2)
        statistics.loc[row, 'STD_'+title] = np.round(np.std(col), 2)

        #statistic, pvalue = stats.kstest(col_1, col_2)
        #statistics.loc[row, 'PVALUE'] = np.round(pvalue, 2)
    

### EXPLICIT

In [ ]:
id_true_pred = [0, 1, 4, 5, 6, 8, 9, 10, 11, 13, 14, 16, 17, 18, 19, 20, 23, 24, 25, 26, 28, 31, 33, 35, 36, 37, 40, 41, 42, 43, 44, 46, 47, 49, 50, 51, 54, 57]
id_younger_pred = [29, 34, 39, 45, 48, 52, 53, 55, 56, 59, 60, 61]
id_older_pred = [2, 3, 7, 12, 15, 21, 22, 27, 30, 32, 38, 58]

update_statistics(df, id_true_pred, "explicit", "TRUE")
update_statistics(df, id_younger_pred, "explicit", "YOUNGER")
update_statistics(df, id_older_pred, "explicit", "OLDER")
statistics

In [ ]:
# Mean of Godspeed sections based age group and gender prediction in the EXPLICIT case
fig, ax = plt.subplots(1,1,figsize=(12, 5)) 

color_true_pred = "g"
color_younger_pred = "b"
color_older_pred = "r"

true = statistics["MEAN_TRUE"].tolist()
ax.plot(sections, true, color_true_pred+'o-', label="Right age group") 

younger = statistics["MEAN_YOUNGER"].tolist()
ax.plot(sections, younger, color_younger_pred+'o-', label="Younger age group") 

older = statistics["MEAN_OLDER"].tolist()
ax.plot(sections, older, color_older_pred+'o-', label="Older age group or wrong gender") 

plt.yticks(np.arange(0, 5.5, 0.5))
plt.grid(axis='x')
plt.ylabel('Mean')
plt.legend(loc='lower left')

plt.savefig(folder_plots+"/explicit.jpg")
plt.show()

### IMPLICIT

In [ ]:
id_true_pred = [3, 4, 5, 6, 8, 9, 10, 12, 13, 15, 16, 17, 19, 20, 23, 24, 25, 28, 30, 31, 33, 37, 38, 40, 41, 42, 43, 44, 46, 47, 50, 51, 54, 58, 60]
id_younger_pred = [0, 18, 26, 29, 34, 36, 39, 45, 48, 52, 53, 55, 56, 59, 61]
id_older_pred = [1, 2, 7, 11, 14, 21, 22, 27, 32, 35, 49, 57]

update_statistics(df, id_true_pred, "implicit", "TRUE")
update_statistics(df, id_younger_pred, "implicit", "YOUNGER")
update_statistics(df, id_older_pred, "implicit", "OLDER")
statistics

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12, 5)) 

color_true_pred = "g"
color_younger_pred = "b"
color_older_pred = "r"

true = statistics["MEAN_TRUE"].tolist()
ax.plot(sections, true, color_true_pred+'o-', label="Right age group") 

younger = statistics["MEAN_YOUNGER"].tolist()
ax.plot(sections, younger, color_younger_pred+'o-', label="Younger age group") 

older = statistics["MEAN_OLDER"].tolist()
ax.plot(sections, older, color_older_pred+'o-', label="Older age group or wrong gender") 

plt.yticks(np.arange(0, 5.5, 0.5))
plt.grid(axis='x')
plt.ylabel('Mean')
plt.legend(loc='lower left')

plt.savefig(folder_plots+"/implicit.jpg")
plt.show()